 # Toyota Used Car Market Mispricing Analysis

 ## From Price Prediction to Market Inefficiency Detection



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib import rcParams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

#mpl.rcParams.update(mpl.rcParamsDefault)  # 重置为默认设置
plt.rcParams['figure.figsize'] = (10, 6)  # 使用更小的尺寸
plt.rcParams['figure.dpi'] = 100  # 降低分辨率
plt.rcParams['font.size'] = 10
# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
# 设置图表样式
sns.set_style("whitegrid")
# 设置图表样式
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
# 读取数据
data = pd.read_csv(r"E:\项目集\丰田二手车市场洞察\toyota.csv")


In [ ]:
# 定义函数：使用IQR检测异常值，并用中位数替换
def replace_outliers_with_median(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    median = df[column].median()
    # 识别异常值
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    num_outliers = outliers.sum()
    # 用中位数替换异常值
    df.loc[outliers, column] = median
    print(f"Replaced {num_outliers} outliers in {column} with median {median}")
    return df

# 先处理明显无效值：engineSize <= 0，用中位数替换（或移除，根据需要）
median_engine = data['engineSize'].median()
data.loc[data['engineSize'] <= 0, 'engineSize'] = median_engine
print(f"Replaced invalid engineSize (<=0) with median {median_engine}")

# 应用到关键数值列：year, mileage, price, tax, mpg, engineSize
columns_to_process = ['year', 'mileage', 'price', 'tax', 'mpg', 'engineSize']
for col in columns_to_process:
    data = replace_outliers_with_median(data, col)

# 检查清洗后数据
print("\nAfter outlier replacement:")
print(data.describe())
print(f"Original rows: 6738, After processing: {len(data)} (no rows removed)")


In [ ]:
data.head()


In [ ]:
data.info()


In [ ]:
data.describe()


In [ ]:
print(data['model'].value_counts())


In [ ]:
print(data['fuelType'].value_counts())


In [ ]:
print(data['transmission'].value_counts())


In [ ]:
print("原始数据中的车型种类:", data['model'].unique())
print("车型数量:", len(data['model'].unique()))

# 检查映射字典的键
model_mapping = {model: idx for idx, model in enumerate(data['model'].unique())}
print("映射字典的键:", model_mapping.keys())
print("映射字典键的数量:", len(model_mapping))


In [ ]:
data1 = data[['price','year','mileage','tax','mpg','engineSize']]


In [ ]:
data1


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data1.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(data['year'], data['price'], alpha=0.5)
plt.xlabel('Year')
plt.ylabel('Price')

plt.subplot(1, 2, 2)
plt.scatter(data['mileage'], data['price'], alpha=0.5)
plt.xlabel('Mileage')
plt.ylabel('Price')
plt.show()


In [ ]:
# 过滤掉数据量太少的车型
model_counts = data['model'].value_counts()
valid_models = model_counts[model_counts >= 5].index  # 只保留至少有5个数据点的车型
filtered_data = data[data['model'].isin(valid_models)]

plt.figure(figsize=(14, 6))
sns.boxplot(data=filtered_data, x='model', y='price')
plt.xticks(rotation=45)
plt.title('Price Distribution by Model (Only models with ≥5 data points)')
plt.tight_layout()
plt.show()


In [ ]:
data_price = data.copy()
numerical_features = [
    'year', 
    'mileage', 
    'tax', 
    'mpg', 
    'engineSize'
]

categorical_features = [
    'fuelType', 
    'transmission', 
    'model'
]

X = data_price[numerical_features + categorical_features]
y = data_price['price']
# 对类别变量做 One-Hot 编码
X = pd.get_dummies(
    X,
    columns=categorical_features,
    drop_first=True
)
X.shape
X.columns[:10]
X.head()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

print("Train R²:", rf.score(X_train, y_train))
print("Test  R²:", rf.score(X_test, y_test))


In [ ]:
# 确保特征名是从 One-Hot 编码后的 X 中获取的

importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)


top_n = 15

plt.figure(figsize=(10,6))
plt.barh(
    importance_df['feature'][:top_n][::-1],
    importance_df['importance'][:top_n][::-1]
)
plt.xlabel('Feature Importance')
plt.title('Most Important Features for Predicting Used Toyota Prices')
plt.tight_layout()
plt.show()

plt.show()


In [ ]:
#因为数据中2020为最新年份，所以计算车龄时用2021减去年份
data_price['age'] = 2020 - data_price['year']

# 选择特征：车龄、里程、油耗、发动机排量、燃油类型、车型
numerical_features = [
    'age', 
    'mileage', 
    'tax', 
    'mpg', 
    'engineSize'
]

categorical_features = [
    'fuelType', 
    'transmission', 
    'model'
]

X = data_price[numerical_features + categorical_features]
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

y = data_price['price']

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用随机森林模型
rf_model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# 用模型预测价格
data_price['predicted_price'] = rf_model.predict(X)


In [ ]:
# 计算保值率（实际价格 / 预测价格）
data_price['mispricing_index'] = data_price['price'] / data_price['predicted_price']


In [ ]:
# 按车型分组，计算平均保值率
residual_by_model = data_price.groupby('model').agg({
    'mispricing_index': 'mean',
    'price': 'mean',
    'age': 'mean',
    'mileage': 'mean'
})

# 筛选样本量足够的车型（>30辆）
residual_by_model = residual_by_model[residual_by_model['mispricing_index'].notna()]
residual_by_model = residual_by_model.sort_values('mispricing_index', ascending=False)

# 添加样本量字段
residual_by_model['sample_size'] = data_price.groupby('model').size()

# 只保留样本量 > 30 的车型
residual_by_model = residual_by_model[residual_by_model['sample_size'] >= 30]

# 展示保值率排名
residual_by_model[['mispricing_index', 'sample_size']].head(10)


In [ ]:
plt.figure(figsize=(10,6))
bars = plt.barh(
    residual_by_model.index,
    residual_by_model['mispricing_index']
)

# 在每个条形上添加数值
for bar in bars:
    width = bar.get_width()  # 获取条形的宽度，即mispricing_index值
    plt.text(width, bar.get_y() + bar.get_height() / 2, 
             f'{width:.4f}',  # 格式化为4位小数
             va='center', ha='left', fontsize=10)

# 添加一条分界线
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.axvline(1.0, linestyle='--', color='gray', alpha=0.7)  # 分界线：1.0表示市场平均保值率
plt.xlabel('Mispricing Index ( > 1 = Higher than Market Average )')
plt.title('不同车型在控制车龄、里程等条件后的相对保值表现')
plt.gca().invert_yaxis()  # 使得最重要的车型显示在顶部
plt.tight_layout()
plt.show()


In [ ]:
# 设置绘图风格
sns.set(style="whitegrid")

plt.figure(figsize=(10,6))
sns.histplot(data_price['mispricing_index'], bins=30, kde=False, color='skyblue')

# 添加 0.85 和 1.15 分界线
plt.axvline(x=0.85, color='red', linestyle='--', label='Undervalued Threshold (0.85)')
plt.axvline(x=1.15, color='green', linestyle='--', label='Overvalued Threshold (1.15)')

plt.title('Distribution of Mispricing Index')
plt.xlabel('Mispricing Index (Actual Price / Predicted Price)')
plt.ylabel('Number of Vehicles')
plt.legend()
plt.show()



In [ ]:
# 统计低估、高估、合理车辆数量
total = len(data_price)
undervalued = len(data_price[data_price['mispricing_index'] < 0.85])
overvalued = len(data_price[data_price['mispricing_index'] > 1.15])
fair = total - undervalued - overvalued

# 打印比例
print(f"Total vehicles: {total}")
print(f"Undervalued (<0.85): {undervalued} ({undervalued/total:.2%})")
print(f"Overvalued (>1.15): {overvalued} ({overvalued/total:.2%})")
print(f"Fairly priced (0.85~1.15): {fair} ({fair/total:.2%})")



In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data_price[data_price['mispricing_index'] < 0.85]['mispricing_index'], bins=15, color='red', label='Undervalued')
sns.histplot(data_price[(data_price['mispricing_index'] >= 0.85) & (data_price['mispricing_index'] <= 1.15)]['mispricing_index'], bins=15, color='blue', label='Fair')
sns.histplot(data_price[data_price['mispricing_index'] > 1.15]['mispricing_index'], bins=15, color='green', label='Overvalued')

plt.title('Mispricing Index by Category')
plt.xlabel('Mispricing Index')
plt.ylabel('Number of Vehicles')
plt.legend()
plt.show()



In [ ]:
# 按车型分组计算平均 mispricing_index
model_mispricing = data_price.groupby('model')['mispricing_index'].mean().sort_values(ascending=False)

# 查看前10高估车型和前10低估车型
print("Top 10 Overvalued Models:")
print(model_mispricing.head(10))
print("\nTop 10 Undervalued Models:")
print(model_mispricing.tail(10))



In [ ]:
# 给颜色分类
colors = ['green' if x < 1 else 'red' for x in model_mispricing.values]

plt.figure(figsize=(12,6))
sns.barplot(x=model_mispricing.index, y=model_mispricing.values, palette=colors)
plt.axhline(1, color='black', linestyle='--', label='Fair Price')
plt.xticks(rotation=45)
plt.ylabel('Average Mispricing Index')
plt.title('Average Mispricing Index by Toyota Model')
plt.legend()
plt.show()



In [ ]:
# 先确保已经有 age 列，如果没有可以创建
data_price['age'] = 2020 - data_price['year']  # 或者用你的数据年份列

# 计算透视表：车型 × 车龄
heatmap_data = data_price.pivot_table(
    index='model', 
    columns='age', 
    values='mispricing_index', 
    aggfunc='mean'
)

# 查看前几行
heatmap_data.head()



In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(
    heatmap_data, 
    annot=True,           # 显示数值
    fmt=".2f",            # 保留两位小数
    cmap="coolwarm",      # 蓝-白-红，低估-合理-高估
    center=1               # 以 1 为中线，颜色中心
)

plt.title('Average Mispricing Index by Model and Vehicle Age')
plt.xlabel('Vehicle Age (Years)')
plt.ylabel('Toyota Model')
plt.show()



In [ ]:
# 新增 buy_signal 列：True 表示建议收购
data_price['buy_signal'] = data_price['mispricing_index'] < 0.85

# 计算建议收购的车辆数量
buy_count = data_price['buy_signal'].sum()
total_count = len(data_price)
print(f"Total vehicles: {total_count}")
print(f"Suggested to buy: {buy_count} ({buy_count/total_count:.2%})")



In [ ]:
# 只有建议收购的车辆计算潜在利润
data_price['potential_profit'] = data_price['predicted_price'] - data_price['price']

# 策略下平均潜在利润
avg_profit = data_price.loc[data_price['buy_signal'], 'potential_profit'].mean()
total_profit = data_price.loc[data_price['buy_signal'], 'potential_profit'].sum()

print(f"Average potential profit per vehicle: £{avg_profit:.2f}")
print(f"Total potential profit if all recommended vehicles are bought: £{total_profit:.2f}")



In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='buy_signal', data=data_price, palette=['red','green'])
plt.xticks([0,1], ['Do Not Buy','Buy'])
plt.ylabel('Number of Vehicles')
plt.title('Vehicle Buy Signal Distribution')
plt.show()



In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data_price.loc[data_price['buy_signal'], 'potential_profit'], bins=30, color='green')
plt.title('Potential Profit Distribution for Suggested Vehicles')
plt.xlabel('Potential Profit (£)')
plt.ylabel('Number of Vehicles')
plt.show()



In [ ]:
profit_by_model = data_price[data_price['buy_signal']].groupby('model')['potential_profit'].mean().sort_values(ascending=False)
print(profit_by_model.head(10))



In [ ]:
profit_by_age = data_price[data_price['buy_signal']].groupby('age')['potential_profit'].mean()
profit_by_age.plot(kind='bar', figsize=(12,6), color='skyblue')
plt.ylabel('Average Potential Profit (£)')
plt.title('Average Potential Profit by Vehicle Age')
plt.show()



  ​市场趋势：市场是否系统性高估或低估某些车型？

In [ ]:
# 过滤主要车型 (>50样本)
main_models = data['model'].value_counts()[data['model'].value_counts() > 50].index
trend_data = data[data['model'].isin(main_models)]

# 按年份/车型平均价格
price_by_year_model = trend_data.groupby(['year', 'model'])['price'].agg(['mean', 'count']).reset_index()

# 线图：价格趋势
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial', 'times new roman']
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(12, 6))
sns.lineplot(data=price_by_year_model, x='year', y='mean', hue='model', marker='o')
plt.title('平均价格随年份变化（按车型）')
plt.xlabel('年份')
plt.ylabel('平均价格(欧)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# 热力图
pivot = price_by_year_model.pivot(index='year', columns='model', values='mean')
plt.figure(figsize=(12, 8))
sns.heatmap(pivot, annot=True, fmt='.0f', cmap='YlGnBu')
plt.title('不同年份车型平均价格 ')
plt.show()

# 洞察表：年份价格分布
year_summary = data.groupby('year')['price'].agg(['mean', 'median', 'min', 'max', 'count'])
print(year_summary)


In [ ]:
plt.figure(figsize=(8,5))
data_price.boxplot(
    column='mispricing_index',
    by='fuelType',
    grid=False
)
plt.axhline(1.0, linestyle='--')
plt.title('Price Index Distribution by Fuel Type')
plt.suptitle('')
plt.ylabel('Price Index')
plt.show()


